In [10]:
import numpy as np
import keras
from keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import json
import os

In [8]:
# Function to load the JSON data and extract relevant information
def load_game_data(json_file_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    games = []
    for game in data:
        game_info = {
            'appid': game['appid'],
            'title': game['title'],
            'price': game['price'],
            'sentiment': game['sentiment'],
            'screenshots': game.get('screenshots', []),
        }
        games.append(game_info)

    return games

In [9]:
# Load the JSON data
json_file_path = 'C:/Users/Beste/Desktop/AAB/dataset.json'
game_data = load_game_data(json_file_path)

In [11]:
# Function to load images
def preprocess_image(screenshot_filename,  target_size=(640, 480)):
    # Load image, resize, and convert to NumPy array
    img = load_img(screenshot_filename,  target_size=target_size)
    img_array = img_to_array(img)

    # Normalize pixel values 
    img_array = img_array / 255.0

    return img_array

In [16]:
screenshots_dir = "C:/Users/Beste/Desktop/AAB/images-big"

X = []  # List to store image data

for game in game_data:
    for screenshot_filename in game['screenshots']:
        screenshot_path = os.path.join(screenshots_dir, screenshot_filename)
        img_array = preprocess_image(screenshot_path)
        X.append(img_array)

MemoryError: Unable to allocate 23.7 MiB for an array with shape (1080, 1920, 3) and data type float32

In [ ]:
# Standardize the dataset RGB and build the model 
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)